In [4]:
import itertools
import heapq
import numpy as np

In [5]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read().split('\n')

In [9]:

class Tile():
    def __init__(self, y, x):
        self.x = x
        self.y = y
        self.parent = None
        self.G = np.inf
        self.F = np.inf
        self.H = np.inf
          
    def __repr__(self):
        return f'({self.y}, {self.x})'

def generate_graph(rowmaze):
    tiles = {(y,x): Tile(y,x) for y,x in itertools.product(range(len(rowmaze)), range(len(rowmaze[0])))}
    graph = {}
    for y,row in enumerate(rowmaze):
        for x,current_height in enumerate(row):
                   
            children = []
            for direction in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares
                new_tile = (y + direction[0],
                            x + direction[1])
                
                if (new_tile[0] > len(rowmaze) - 1) or (new_tile[0] < 0) or (new_tile[1] > len(rowmaze[len(rowmaze)-1]) - 1) or (new_tile[1] < 0):
                    continue
                
                node_height = rowmaze[new_tile[0]][new_tile[1]]
                if (ord(node_height) - ord(current_height)) > 1:
                    continue
                
                children.append(tiles[new_tile[0], new_tile[1]])
            
            graph.update({tiles[(y,x)]: children})
    return graph, tiles

            
def aStar(graph, current, end):
    openSet = set()
    openHeap = []
    
    def get_heuristic(t):
        return abs(end.x - t.x) + abs(end.y - t.y)
    
    def retracePath(c):
        path = [c]
        while c.parent is not None:
            c = c.parent
            path.append(c)
        path.reverse()
        return path
    
    current.G = 0
    current.H = get_heuristic(current)
    current.F = current.G + current.H
    
    openSet.add(current)
    openHeap.append((0, id(current), current))
    
    while openSet:
        current = heapq.heappop(openHeap)[2]
        if current == end:
            return retracePath(current)
        
        openSet.remove(current)
        for tile in graph[current]:
            
            tentative_g = current.G + 1                
            if tentative_g < tile.G:
                tile.parent = current
                tile.G = tentative_g
                tile.H = get_heuristic(tile)
                tile.F = tile.G + tile.H
                if tile not in openSet:
                    openSet.add(tile)
                    heapq.heappush(openHeap, (tile.F, id(tile), tile))
    return []

def find_point(pt,rowmaze):
    for irow,row in enumerate(rowmaze):
        try:
            j = row.index(pt)
            i = irow
        except ValueError:
            continue
    return i,j

## Part 1

In [10]:
rowmaze = get_input('input')

start = find_point('S', rowmaze)
end = find_point('E', rowmaze)

rowmaze[start[0]] = rowmaze[start[0]].replace('S', 'a')
rowmaze[end[0]] = rowmaze[end[0]].replace('E', 'z')
graph, tiles = generate_graph(rowmaze)
start_tile = tiles[start]
end_tile = tiles[end]

path = aStar(graph, start_tile, end_tile)
print(len(path)-1)

517


## Part 2

In [11]:
rowmaze = get_input('input')

start = find_point('S', rowmaze)
end = find_point('E', rowmaze)

rowmaze[start[0]] = rowmaze[start[0]].replace('S', 'a')
rowmaze[end[0]] = rowmaze[end[0]].replace('E', 'z')

minimum_path_length = np.inf
for y in range(len(rowmaze)):
    graph, tiles = generate_graph(rowmaze)
    start_tile = tiles[(y, 0)]
    end_tile = tiles[end]
    path = aStar(graph, start_tile, end_tile)
    if path:
        minimum_path_length = min(len(path), minimum_path_length)
print(minimum_path_length - 1)

512


## Trying with NetworkX

In [13]:
import networkx as nx

def generate_graph(rowmaze):
    graph = nx.DiGraph()
    for y,row in enumerate(rowmaze):
        for x,current_height in enumerate(row):
            
            graph.add_node((y,x))       
            for direction in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares
                new_tile = (y + direction[1],
                            x + direction[0])
                
                if (new_tile[0] > len(rowmaze) - 1) or (new_tile[0] < 0) or (new_tile[1] > len(rowmaze[0]) - 1) or (new_tile[1] < 0):
                    continue
                
                node_height = rowmaze[new_tile[0]][new_tile[1]]
                if (ord(node_height) - ord(current_height)) > 1:
                    continue
                
                graph.add_edge( (y,x), (new_tile[0], new_tile[1]) )
    return graph

def dist(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

### Part 1

In [15]:
rowmaze = get_input('input')

start = find_point('S', rowmaze)
end = find_point('E', rowmaze)

rowmaze[start[0]] = rowmaze[start[0]].replace('S', 'a')
rowmaze[end[0]] = rowmaze[end[0]].replace('E', 'z')
G = generate_graph(rowmaze)

nx.astar_path_length(G, start, end, heuristic=dist)

517

### Part 2

In [17]:
rowmaze = get_input('input')

start = find_point('S', rowmaze)
end = find_point('E', rowmaze)

rowmaze[start[0]] = rowmaze[start[0]].replace('S', 'a')
rowmaze[end[0]] = rowmaze[end[0]].replace('E', 'z')
G = generate_graph(rowmaze)

sources = [(y,0) for y in range(len(rowmaze))]
length, _ = nx.multi_source_dijkstra(G, sources, end)
print(length)

512
